# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [26]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
import pymorphy2
import pandas as pd
from functools import reduce

# Загрузка необходимых ресурсов NLTK
nltk.download('wordnet')
nltk.download('stopwords')

# Инициализация стеммера для русского языка
stemmer = SnowballStemmer("russian")


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [12]:
# Случайное число n в диапазоне от 301 до 500
n = random.randint(301, 500)

# Загрузка данных из файла preprocessed_descriptions.csv и выбор первых n строк
preprocessed_descriptions = pd.read_csv("./data/preprocessed_descriptions.csv")[:n]

# Вывод первых нескольких строк для проверки
preprocessed_descriptions.head()

# Объединяем токенизированные слова из столбца 'preprocessed_descriptions'
words = reduce(lambda x, y: x + y, [word_tokenize(item) for item in preprocessed_descriptions["preprocessed_descriptions"].to_list() if isinstance(item, str)])

# Вывод первых 10 слов для проверки
words[:10]

['an',
 'original',
 'recipe',
 'created',
 'by',
 'chef',
 'scott',
 'meskan',
 'george',
 's']

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [13]:
# Преобразуем список слов во множество, чтобы удалить дубликаты, затем снова преобразуем обратно в список
words = list(set(words))

# Создаем список из 5 пар случайных слов
pairs = [' '.join(random.choices(words, k=2)) for _ in range(5)]

# Выводим список пар слов
print(pairs)

# Вычисляем расстояние Левенштейна для каждой пары слов
h_s = [edit_distance(*v.split()) for v in pairs]

# Выводим расстояния Левенштейна
h_s


['same until', 'finding elephant', 'sink spinach', 'mine creaks', 'thaw minor']


[5, 7, 4, 6, 5]

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [15]:
def k_nearest(word: str, k: int = 1):
    # Сортируем слова по расстоянию Левенштейна до заданного слова
    w_new = sorted(words, key=lambda w: edit_distance(w, word))
    # Возвращаем k ближайших слов
    return w_new[:k]

# Пример вызова функции k_nearest для слова 'check' и получения 3 ближайших слов
k_nearest('check', k=3)


['check', 'chuck', 'cheap']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [18]:
lemmatizer = WordNetLemmatizer()

# Применяем стеммер к каждому слову в списке words
stemmed_words = [stemmer.stem(word) for word in words]

# Применяем лемматизатор к каждому слову в списке words
normalized_words = [lemmatizer.lemmatize(word) for word in words]

# Создаем DataFrame с данными и выбираем строки с индексами от 10 до 19
df = pd.DataFrame(list(zip(words, stemmed_words, normalized_words))[10:20], columns=['word', 'stemmed_word', 'normalized_word'])

# Устанавливаем столбец 'stemmed_word' в качестве индекса
df = df.set_index('stemmed_word')

# Выводим первые несколько строк DataFrame
df.head()


,word,normalized_word
stemmed_word,,
apples,apples,apple
temptations,temptations,temptation
maria,maria,maria
org,org,org
whipped,whipped,whipped


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [22]:
```python
st_w = stopwords.words()

# Объединяем все тексты из столбца 'preprocessed_descriptions' в одну строку
texts = reduce(lambda x, y: x + ' ' + y, [item for item in preprocessed_descriptions["preprocessed_descriptions"].to_list() if isinstance(item, str)])

# Токенизируем тексты
tokens = word_tokenize(texts)

# Общее количество токенов
l = len(tokens)

# Инициализируем счетчики слов
l_new = 0
words_dict = {}
words_dict_stop = {}

# Подсчитываем количество вхождений каждого слова, включая стоп-слова
for w in tokens:
    words_dict[w] = words_dict.get(w, 0) + 1
    if w not in st_w:
        words_dict_stop[w] = words_dict_stop.get(w, 0) + 1
        l_new += 1

# Доля стоп-слов в тексте
stopword_ratio = (l - l_new) / l

# Выводим результаты
print(f'Доля стоп-слов - {stopword_ratio}')
print(f'Топ 10 слов со стоп-словами: {"; ".join(sorted(words_dict.keys(), key=lambda x: words_dict[x], reverse=True)[:10])}')
print(f'Топ 10 слов без стоп-слов: {"; ".join(sorted(words_dict_stop.keys(), key=lambda x: words_dict_stop[x], reverse=True)[:10])}')
```

Доля стоп слов - 0.5844606728121633
Топ 10 слов со стоп словами: the; a; i; and; this; it; to; is; of; for
Топ 10 слов без стоп слов: recipe; make; great; time; easy; dish; made; bread; delicious; soup


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [30]:
data = preprocessed_descriptions.sample(5)
print(data)

# Инициализируем TF-IDF векторизатор
vectorizer = TfidfVectorizer()

# Обучаем векторизатор на данных из столбца 'preprocessed_descriptions'
vectorizer.fit(data['preprocessed_descriptions'])

# Преобразуем тексты в TF-IDF векторы
sent_vec = vectorizer.transform(data['preprocessed_descriptions'])

# Преобразуем разреженные матрицы TF-IDF векторов в массивы numpy
sent_vec = sent_vec.toarray()

# Выводим оригинальный текст и соответствующий ему TF-IDF вектор
for i, recipe in enumerate(data['preprocessed_descriptions']):
    print("Рецепт:\n", recipe)
    print("Вектор:\n", sent_vec[i])
    print()


                                       name  \
226  50   whole wheat bread   bread machine   
111   10 layer poor man s lasagna casserole   
392  aguado de gallina or chicken rice soup   
247                        a   w cream soda   
359                           afghan monkey   

                             preprocessed_descriptions  
226  this is a nice light bread that is very tasty ...  
111  i had to come up with a kid adult friendly rec...  
392  this soup looks delicious  i found the recipe ...  
247          cooking time does not include chill time   
359  a great combination of coconut rum  melon   ba...  
Рецепт:
 this is a nice light bread that is very tasty for sandwiches and does not require the whole wheat cycle on the abm  it is from the  canadian bread machine baking  with roxanne mcquilkin 
Вектор:
 [0.         0.1673716  0.         0.         0.11209071 0.
 0.1673716  0.         0.         0.         0.33474319 0.
 0.1673716  0.         0.         0.         0.     

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [32]:
import itertools
from scipy.spatial import distance

# Инициализация переменных для хранения наибольшего коэффициента и пары текстов с наибольшим сходством
max_pair = None
max_result = -1

# Создаем словарь для хранения коэффициентов сходства между текстами
coeff_dict = {}

# Инициализируем TfidfVectorizer для анализа слов и удаления стоп-слов на английском языке
vectorizer3 = TfidfVectorizer(analyzer="word", stop_words="english")
transform3 = vectorizer3.fit_transform(data["preprocessed_descriptions"].to_list())

# Создаем список, содержащий тексты и их TF-IDF матрицы
all_data = list(zip(data["preprocessed_descriptions"].to_list(), transform3.toarray()))

# Проходим по всем возможным парам текстов и вычисляем коэффициент сходства
for pair in itertools.product(all_data, repeat=2):
    text1, matrix1 = pair[0]
    text2, matrix2 = pair[1]
    
    # Вычисляем косинусное расстояние между TF-IDF векторами текстов
    result = distance.cosine(matrix1, matrix2)
    
    # Вычисляем обратное косинусное расстояние для получения коэффициента сходства
    inverse_result = 1 - result
    
    # Добавляем коэффициент сходства в словарь для соответствующего текста
    if text1 not in coeff_dict:
        coeff_dict[text1] = []
    coeff_dict[text1].append(inverse_result)

    # Обновляем наибольший коэффициент и пару текстов с наибольшим сходством
    if inverse_result > max_result and text1 != text2:
        max_result = inverse_result
        max_pair = (text1, text2)
    
    # Выводим тексты и их коэффициенты сходства для отладки
    print(f"{text1}\n{text2}\n{inverse_result}\n")

# Создаем DataFrame из словаря коэффициентов сходства
df_final2 = pd.DataFrame.from_dict(coeff_dict)
df_final2.columns = data["preprocessed_descriptions"].to_list()
df_final2.index = data["preprocessed_descriptions"].to_list()
df_final2


this is a nice light bread that is very tasty for sandwiches and does not require the whole wheat cycle on the abm  it is from the  canadian bread machine baking  with roxanne mcquilkin 
this is a nice light bread that is very tasty for sandwiches and does not require the whole wheat cycle on the abm  it is from the  canadian bread machine baking  with roxanne mcquilkin 
1.0

this is a nice light bread that is very tasty for sandwiches and does not require the whole wheat cycle on the abm  it is from the  canadian bread machine baking  with roxanne mcquilkin 
i had to come up with a kid adult friendly recipe to please everybody while my step daughter was visiting  so this is what i did  we all loved this  it ended up making a lot  though  so you might want to make a half a recipe  i ll be taking this to the next potluck or family gathering that comes up  it was so good  it came out way better than i thought it would  i was quite surprised and happy  hope you try it 
0.0

this is a nice

,this is a nice light bread that is very tasty for sandwiches and does not require the whole wheat cycle on the abm it is from the canadian bread machine baking with roxanne mcquilkin,i had to come up with a kid adult friendly recipe to please everybody while my step daughter was visiting so this is what i did we all loved this it ended up making a lot though so you might want to make a half a recipe i ll be taking this to the next potluck or family gathering that comes up it was so good it came out way better than i thought it would i was quite surprised and happy hope you try it,this soup looks delicious i found the recipe at www laylita com recipes posted for zwt 2008,cooking time does not include chill time,a great combination of coconut rum melon banana and pineapple
this is a nice light bread that is very tasty for sandwiches and does not require the whole wheat cycle on the abm it is from the canadian bread machine baking with roxanne mcquilkin,1.000000,0.000000,0.000000,0.056012,0.0
i had to come up with a kid adult friendly recipe to please everybody while my step daughter was visiting so this is what i did we all loved this it ended up making a lot though so you might want to make a half a recipe i ll be taking this to the next potluck or family gathering that comes up it was so good it came out way better than i thought it would i was quite surprised and happy hope you try it,0.000000,1.000000,0.067811,0.000000,0.0
this soup looks delicious i found the recipe at www laylita com recipes posted for zwt 2008,0.000000,0.067811,1.000000,0.000000,0.0
cooking time does not include chill time,0.056012,0.000000,0.000000,1.000000,0.0
a great combination of coconut rum melon banana and pineapple,0.000000,0.000000,0.000000,0.000000,1.0


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).

In [33]:
# Вывод информации о наибольшем количестве совпадений в предложениях из датасета

# Выводим текст с наибольшим количеством совпадений в предложениях
print(f"Из датасета выше больше всего совпадений в предложениях:\n\n{max_pair[0]}\n\n{max_pair[1]}\n\n{max_result}")


Из датасета выше больше всего совпадений в предложениях:

i had to come up with a kid adult friendly recipe to please everybody while my step daughter was visiting  so this is what i did  we all loved this  it ended up making a lot  though  so you might want to make a half a recipe  i ll be taking this to the next potluck or family gathering that comes up  it was so good  it came out way better than i thought it would  i was quite surprised and happy  hope you try it 

this soup looks delicious  i found the recipe at www laylita com recipes  posted for zwt 2008 

0.06781105003700483
